![Banner logo](../fig/citrine_banner.png)

# Data Format to PIF Converter

*Authors: Carena Church, Enze Chen*

Let's write an converter to map the Instron output file to a PIF file.

## Python package imports

In [ ]:
# Standard packages
from os import environ
from uuid import uuid4

# Third-party packages
from thermo_mechanical_analysis import Thermo
from pypif import pif
from pypif.obj import *
from citrination_client import CitrinationClient

## Read the Instron files
We will first read in and print the Instron output file.

In [ ]:
with open("RT-loadtofail_10e-3.txt") as f:
    lines = f.readlines()
    print('\n'.join(lines[:15]))

## `instron_to_pif`

Using a given analysis script to compile the force, displacement, stress, and strain, and using those values to calculate the elastic modulus and critical stress (thanks Saurabh!), we can package the Instron output into the PIF format.

In [ ]:
def instron_to_pif(instron_file, material_name):
    instron = Thermo(instron_file)
    instron.extract_data()
    instron.calc_elastic_modulus()
    system = ChemicalSystem(name=material_name)

    elastic_modulus = Property(name="Elastic Modulus", units="MPa", scalars=instron.elastic_modulus)
    critical_stress = Property(name="Critical Stress", units="MPa", scalars=instron.critical_stress)
    stress_strain = Property(name="Stress", units="MPa", scalars=instron.stress,
                             conditions=Value(name="Strain", units="mm/mm", scalars=instron.strain))
    force_displacement = Property(name="Force", units="N", scalars=instron.force,
                                  conditions=Value(name="Displacement", units="mm", scalars=instron.displacement))
    system.properties = [elastic_modulus, critical_stress, stress_strain, force_displacement]
    return system

Now, upload the result to Citrination.

In [ ]:
# Instantiate the CitrinationClient
site = 'https://citrination.com'
client = CitrinationClient(api_key=environ['CITRINATION_API_KEY'], site=site)

# Create a dataset with random name
dataset_name = "Tutorial dataset " + str(uuid4())[:6]
dataset = client.data.create_dataset(name=dataset_name, description="Dataset for Instron tutorial.", public=False)
dataset_id = dataset._id
print('Dataset created! {}/datasets/{}'.format(site, dataset_id))

In [ ]:
instron = instron_to_pif("RT-loadtofail_10e-3.txt", "Sample")
instron.uid = "instron_example"

with open("instron_pif.json", 'w') as fp:
    pif.dump(instron, fp)
    
res = client.data.upload(dataset_id, "instron_pif.json")

Soon you should begin to the see the dataset populated by PIFs!